In [1]:
%cd ..

/home/pauline/Dokumente/Delab_Projekt/delab


In [2]:
import pandas as pd

In [4]:
from util.sql_switch import get_query_native

df = get_query_native(
    "SELECT author_id, conversation_id, name , is_climate_author, climate_author_type FROM delab_tweet, delab_tweetauthor WHERE delab_tweet.author_id=delab_tweetauthor.twitter_id")
df2 = df.loc[df['is_climate_author']== True]
df3 = get_query_native("SELECT * FROM delab_conversation")
df4 = pd.merge(df2,df3, on='conversation_id')
df4.tail(20)

using postgres
using postgres


/home/pauline/Dokumente/Delab_Projekt/delab/venv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/pauline/Dokumente/Delab_Projekt/delab/venv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,author_id,conversation_id,name,is_climate_author,climate_author_type,id,depth,branching_factor,root_dominance
47,825306344961998848,1595682559459201024,Volksverpetzer,True,non-governmental organisation,26,12,48.0,49.000000
48,3459051,1584786835737395201,Greenpeace,True,non-governmental organisation,18,2,6.0,0.142857
49,3459051,1581701274168627200,Greenpeace,True,non-governmental organisation,10,2,7.0,0.125000
50,1052189708195782656,1596783546370170881,Annika Joeres,True,journalist,28,5,43.0,44.000000
51,1052189708195782656,1596783546370170881,Annika Joeres,True,journalist,28,5,43.0,44.000000
52,1052189708195782656,1596783546370170881,Annika Joeres,True,journalist,28,5,43.0,44.000000
53,253998571,1596783546370170881,Stefan Rahmstorf 🌏 fediscience.org/@rahmstorf 🦣,True,scientist,28,5,43.0,44.000000
54,1052189708195782656,1594989962193141763,Annika Joeres,True,journalist,30,5,19.0,20.000000
55,1052189708195782656,1594989962193141763,Annika Joeres,True,journalist,30,5,19.0,20.000000
56,1052189708195782656,1594989962193141763,Annika Joeres,True,journalist,30,5,19.0,20.000000
